In [ ]:
from __future__ import print_function  

In [ ]:
import numpy as np
import json
import os 

from keras.preprocessing import sequence


In [ ]:
sequences_file = os.path.join('..', 'data', 'protein-seqs-2018-01-16-131956.txt')
functions_file = os.path.join('..', 'data', 'protein-functions-2018-01-16-131956.txt')

In [ ]:
with open(functions_file) as fn_file:
    has_function = json.load(fn_file)

In [ ]:
has_function  # just to see what we have loaded 

In [ ]:
max_sequence_size = 500   # any sequence longer than this, we ignore (just for now) 

In [ ]:
X = []           # sequences in the same order corresponding to elements of p 
y = []           # output class: 1 if protein has the function, 0 if not 

In [ ]:
# for seeing how many examples we've found for each class 
pos_examples = 0
neg_examples = 0   

In [ ]:
with open(sequences_file) as f:
    for line in f:
        ln = line.split(',')
        protein_id = ln[0].strip()
        seq = ln[1].strip()

        # we're doing this to reduce input size
        if len(seq) >= max_sequence_size:
            continue
        
        print(line)
        
        X.append(seq)
        
        if protein_id in has_function: 
            y.append(1) 
            pos_examples += 1 
        else: 
            y.append(0) 
            neg_examples += 1 

In [ ]:
print("Positive Examples: %d" % pos_examples)
print("Negative Examples: %d" % neg_examples)  # Total is different because we ignored longer sequences 

In [ ]:
def sequence_to_indices(sequence):
    """Convert amino acid letters to indices. 
       _ means no amino acid (used for padding to accommodate for variable length)"""
    
    try:
        acid_letters = ['_', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',
                'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']

        indices = [acid_letters.index(c) for c in list(sequence)]
        return indices
    except Exception:
        print(sequence)
        raise Exception

In [ ]:
sequence_to_indices('AD')  # just testing 

In [ ]:
X_all = [] 
for i in range(len(X)): 
    x = sequence_to_indices(X[i])
    X_all.append(x) 
    

In [ ]:
X_all = np.array(X_all)
y_all = np.array(y)

In [ ]:
print(y[0])
print(X_all[0])
print(len(X_all[0]))

In [ ]:
X_all = sequence.pad_sequences(X_all, maxlen=max_sequence_size)  # to overcome the variable length issue 

In [ ]:
X_all[0]

# Now we need to split the data 

In [ ]:
print(X_all.shape)  # extremely important that you view this! 
print(y_all.shape)  # make sure you are comfortable with shapes! 

We'll do a basic shuffle and 66%, 33% split. 

In [ ]:
n = X_all.shape[0]  # number of data points 

In [ ]:
# randomize to shuffle first
randomize = np.arange(n)
np.random.shuffle(randomize)

In [ ]:
randomize

In [ ]:
X_all = X_all[randomize]
y_all = y_all[randomize]

In [ ]:
test_split = round(n * 2 / 3)
X_train = X_all[:test_split]   # start to (just before) test_split 
y_train = y_all[:test_split]   
X_test  = X_all[test_split:]   # test_split to end 
y_test  = y_all[test_split:]

In [ ]:
# Print shapes again 
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Understanding the Shapes 

In [ ]:
"     [  .  .  .  .  .  5 ]         Initial shape: (5, )   "


In [ ]:
"""     [ [. . . . . . . . 500 ]    []   []   []   []  ]      

Shape is now: (5, 500)
""""""

In [ ]:
# This, when converted to one-hot representation becomes: 

""" 
[ [
   [. . . . . . . . . . 23]             (where 23 is the number of amino acids)
    . 
    .
    .
    500 
  ]    
  []   
  []   
  []   
  []  
  ] 
  
  So, the final shape will be: (5, 500, 23)
"""

# The Model 

In [ ]:
from keras.layers import Embedding, Input, Dropout, Flatten, Dense, Activation
from keras.models import Model, Sequential
from keras.optimizers import SGD

In [ ]:
num_amino_acids = 23 
embedding_dims = 10 
nb_epoch = 2
batch_size = 2

In [ ]:
model = Sequential() 

model.add(Embedding(num_amino_acids, embedding_dims, input_length=max_sequence_size  ))
model.add(Flatten())
model.add(Dense(25, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(),
metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
hist = model.fit(X_train, y_train,
                  batch_size = batch_size,
                  epochs = nb_epoch, 
                  validation_data = (X_test, y_test),
                  verbose=1)   

# Changing to the Functional API 

In [ ]:
input = Input(shape=(max_sequence_size,))

In [ ]:
embedding = Embedding(num_amino_acids, embedding_dims)(input)

In [ ]:
x = Flatten()(embedding)
x = Dense(25, activation='sigmoid')(x)
x = Dense(1)(x)

In [ ]:
output = Activation('sigmoid')(x)

In [ ]:
model = Model([input], output)
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train,
                  batch_size = batch_size,
                  epochs = nb_epoch, 
                  validation_data = (X_test, y_test),
                  verbose=1)    

In [ ]:
hist.history